<a href="https://colab.research.google.com/github/RishiP2006/ML-Trading-Model/blob/main/ML_Trading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

import yfinance as yf


In [2]:
# Load Nifty50 historical data
nifty = yf.download("^NSEI", start="2015-01-01", end="2024-12-31", auto_adjust=False)

# Create technical indicators (features)
nifty["Returns"] = nifty["Adj Close"].pct_change()
nifty["SMA_10"] = nifty["Adj Close"].rolling(window=10).mean()
nifty["SMA_50"] = nifty["Adj Close"].rolling(window=50).mean()
nifty["Volatility"] = nifty["Returns"].rolling(window=10).std()

# Drop NaN values
nifty.dropna(inplace=True)


[*********************100%***********************]  1 of 1 completed


In [3]:
# Define target variable (1 = Buy, -1 = Sell, 0 = Hold)
nifty["Signal"] = 0
nifty.loc[nifty["SMA_10"] > nifty["SMA_50"], "Signal"] = 1  # Buy
nifty.loc[nifty["SMA_10"] < nifty["SMA_50"], "Signal"] = -1  # Sell


In [4]:
# Prepare data
features = ["SMA_10", "SMA_50", "Volatility"]
X = nifty[features]
y = nifty["Signal"]

# Train-test split (Train: 2015-2022, Test: 2024)
train_data = nifty.loc["2015":"2022"]
test_data = nifty.loc["2024"]

X_train, X_test = train_data[features], test_data[features]
y_train, y_test = train_data["Signal"], test_data["Signal"]

# Scale data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train RandomForest model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

# Make predictions
y_pred = model.predict(X_test_scaled)

# Evaluate performance
print(classification_report(y_test, y_pred))

# Add predictions to test data
test_data["Predicted_Signal"] = y_pred


              precision    recall  f1-score   support

          -1       0.23      0.29      0.25        48
           1       0.81      0.76      0.78       197

    accuracy                           0.67       245
   macro avg       0.52      0.52      0.52       245
weighted avg       0.70      0.67      0.68       245



<ipython-input-4-06276f152d22>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data["Predicted_Signal"] = y_pred


In [12]:
# Initialize trading parameters
initial_capital = 100000  # Starting capital
capital = initial_capital
position = 0  # No open positions initially
trade_log = []  # List to store trade history

# Risk Management Settings
risk_per_trade = 0.2  # Allocate only 20% of capital per trade
stop_loss_pct = 0.01  # Stop loss at 1% below purchase price
take_profit_pct = 0.05  # Take profit at 5% above purchase price
max_drawdown_pct = 0.25  # Stop trading if capital drops 25% from peak

# Track highest capital to enforce max drawdown
highest_capital = capital

# Simulate trading
for i in range(1, len(test_data)):
    try:
        predicted_signal = test_data["Predicted_Signal"].iloc[i]

        # Ensure it's a single integer value
        if isinstance(predicted_signal, (pd.Series, np.ndarray)):
            predicted_signal = predicted_signal.iloc[0]  # Take first value if it's a Series
        predicted_signal = int(predicted_signal)  # Convert to integer

        # Ensure close price is a float without triggering a warning
        close_price = test_data["Adj Close"].iloc[i].item()

        # Check max drawdown limit
        if capital < highest_capital * (1 - max_drawdown_pct):
            print("Max drawdown reached. Stopping trading.")
            break

        # Trading logic with risk management
        if predicted_signal == 1 and position == 0:  # Buy
            trade_capital = capital * risk_per_trade  # Only use a portion of capital
            position = trade_capital / close_price
            capital -= trade_capital
            entry_price = close_price  # Store entry price for stop-loss/take-profit tracking
            trade_log.append(("Buy", test_data.index[i], close_price))

        elif predicted_signal == -1 and position > 0:  # Sell
            capital += position * close_price
            position = 0
            trade_log.append(("Sell", test_data.index[i], close_price))

        # Risk management: Check stop-loss & take-profit
        if position > 0:
            if close_price <= entry_price * (1 - stop_loss_pct):
                capital += position * close_price  # Sell at stop-loss price
                position = 0
                trade_log.append(("Stop-Loss Sell", test_data.index[i], close_price))

            elif close_price >= entry_price * (1 + take_profit_pct):
                capital += position * close_price  # Sell at take-profit price
                position = 0
                trade_log.append(("Take-Profit Sell", test_data.index[i], close_price))

        # Update highest capital to track drawdown
        highest_capital = max(highest_capital, capital)

    except Exception as e:
        print(f"Error at index {i}: {e}")  # Debugging if something goes wrong

# Ensure final position is closed
if position > 0:
    capital += position * test_data["Adj Close"].iloc[-1].item()
    position = 0
    trade_log.append(("Final Sell", test_data.index[-1], test_data["Adj Close"].iloc[-1].item()))

# Print final capital after all trades
print(f"Final Capital: {capital}")
print(f"Number of Trades: {len(trade_log)}")

# Show trade log
for trade in trade_log:
    print(trade)


Final Capital: 101138.2864186007
Number of Trades: 42
('Buy', Timestamp('2024-01-02 00:00:00'), 21665.80078125)
('Sell', Timestamp('2024-01-17 00:00:00'), 21571.94921875)
('Buy', Timestamp('2024-02-07 00:00:00'), 21930.5)
('Sell', Timestamp('2024-02-08 00:00:00'), 21717.94921875)
('Buy', Timestamp('2024-02-12 00:00:00'), 21616.05078125)
('Sell', Timestamp('2024-03-13 00:00:00'), 21997.69921875)
('Buy', Timestamp('2024-03-14 00:00:00'), 22146.650390625)
('Sell', Timestamp('2024-03-15 00:00:00'), 22023.349609375)
('Buy', Timestamp('2024-03-18 00:00:00'), 22055.69921875)
('Stop-Loss Sell', Timestamp('2024-03-19 00:00:00'), 21817.44921875)
('Buy', Timestamp('2024-03-20 00:00:00'), 21839.099609375)
('Take-Profit Sell', Timestamp('2024-05-23 00:00:00'), 22967.650390625)
('Buy', Timestamp('2024-05-24 00:00:00'), 22957.099609375)
('Stop-Loss Sell', Timestamp('2024-05-29 00:00:00'), 22704.69921875)
('Buy', Timestamp('2024-05-30 00:00:00'), 22488.650390625)
('Sell', Timestamp('2024-06-03 00:00:0